<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Klassiske_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.6 MB/s eta 0:00:00


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [4]:
# Modeller
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [5]:
# Hyperoptimalisering
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
#optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [6]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [7]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts

In [8]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [9]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_1
from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_2

# Importering av relevant data

In [10]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

# Lagring av resultater

In [11]:
# Oppretter instans for lagring av resultater
res = Resultat_dicts()

# Parametere til hyperoptimalisering

In [12]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
n_gjentagelser = 5

Tilfeldighetsfrø

In [13]:
#Tilfeldighetsfrø
random_seed = 2024

# Testandel og treningsandel

In [15]:
# Setter fast testandel og fast andel av avsatt treningsandel
test_andel = 0.2
trenings_andel =1.0

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: 25% Testsett og Hele resterende treningssett

In [16]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

In [17]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt,
                             dict_res=res.dict_res_1_hel,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-05-01 11:14:45,889] A new study created in memory with name: no-name-1210bc6c-cb95-49f0-a59e-a63f1d08a340
[I 2024-05-01 11:14:46,136] Trial 0 finished with value: 4.408354779819278 and parameters: {'n_neighbors': 12, 'weights': 'distance', 'p': 10}. Best is trial 0 with value: 4.408354779819278.
[I 2024-05-01 11:14:46,166] Trial 1 finished with value: 4.415718799505095 and parameters: {'n_neighbors': 2, 'weights': 'distance', 'p': 8}. Best is trial 0 with value: 4.408354779819278.
[I 2024-05-01 11:14:46,400] Trial 2 finished with value: 4.4563895949013785 and parameters: {'n_neighbors': 2, 'weights': 'uniform', 'p': 4}. Best is trial 0 with value: 4.408354779819278.
[I 2024-05-01 11:14:46,447] Trial 3 finished with value: 4.48790637542236 and parameters: {'n_neighbors': 6, 'weights': 'uniform', 'p': 8}. Best is trial 0 with value: 4.408354779819278.
[I 2024-05-01 11:14:46,716] Trial 4 finished with value: 4.257013855541446 and parameters: {'n_neighbors': 11, 'weights': 'distanc

Lagring av resultater

In [18]:
res_1_hel = pd.DataFrame(res.dict_res_1_hel)
res_1_hel_opt = pd.DataFrame(res.dict_res_1_hel_opt)

res_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_hel.csv')
res_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_hel_opt.csv')

Velger beste modell

## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [20]:
# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

In [21]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt,
                             dict_res=res.dict_res_2_hel,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-05-01 11:32:51,596] A new study created in memory with name: no-name-0567ffa7-ea45-40f3-ae60-a44a7d42917b
[I 2024-05-01 11:32:52,474] Trial 0 finished with value: 5.155926300332452 and parameters: {'n_neighbors': 18, 'weights': 'distance', 'p': 10}. Best is trial 0 with value: 5.155926300332452.
[I 2024-05-01 11:32:52,482] Trial 1 finished with value: 4.81226984910519 and parameters: {'n_neighbors': 15, 'weights': 'distance', 'p': 8}. Best is trial 1 with value: 4.81226984910519.
[I 2024-05-01 11:32:54,306] Trial 3 finished with value: 4.187833924641045 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'p': 2}. Best is trial 3 with value: 4.187833924641045.
[I 2024-05-01 11:32:54,310] Trial 2 finished with value: 5.364840574707749 and parameters: {'n_neighbors': 17, 'weights': 'uniform', 'p': 10}. Best is trial 3 with value: 4.187833924641045.
[I 2024-05-01 11:32:55,739] Trial 5 finished with value: 4.293077919582408 and parameters: {'n_neighbors': 6, 'weights': 'unifor

Lagring av resultater

In [22]:
res_2_hel = pd.DataFrame(res.dict_res_2_hel)
res_2_hel_opt = pd.DataFrame(res.dict_res_2_hel_opt)

res_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_hel.csv')
res_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_hel_opt.csv')

Velger beste modell

## Designproduksjon

## Alternativ 1: 20% Testsett og Hele resterende treningssett

In [23]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

In [24]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt,
                             dict_res=res.dict_res_1_design,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-05-01 11:37:09,253] A new study created in memory with name: no-name-581053b5-c498-4712-a256-3c4f135bb2b7
[I 2024-05-01 11:37:09,442] Trial 1 finished with value: 4.958575421848693 and parameters: {'n_neighbors': 2, 'weights': 'distance', 'p': 7}. Best is trial 1 with value: 4.958575421848693.
[I 2024-05-01 11:37:09,588] Trial 2 finished with value: 4.479073217261023 and parameters: {'n_neighbors': 7, 'weights': 'uniform', 'p': 9}. Best is trial 2 with value: 4.479073217261023.
[I 2024-05-01 11:37:09,595] Trial 0 finished with value: 4.7140712413306565 and parameters: {'n_neighbors': 16, 'weights': 'uniform', 'p': 10}. Best is trial 2 with value: 4.479073217261023.
[I 2024-05-01 11:37:09,800] Trial 3 finished with value: 4.642791080777212 and parameters: {'n_neighbors': 15, 'weights': 'uniform', 'p': 9}. Best is trial 2 with value: 4.479073217261023.
[I 2024-05-01 11:37:09,827] Trial 4 finished with value: 4.4300807724523485 and parameters: {'n_neighbors': 11, 'weights': 'unifo

Lagring av resultater

In [25]:
res_1_design = pd.DataFrame(res.dict_res_1_design)
res_1_design_opt = pd.DataFrame(res.dict_res_1_design_opt)

res_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_design.csv')
res_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_design_opt.csv')

Velger beste modell

## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [26]:
# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

In [27]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt,
                             dict_res=res.dict_res_2_design,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-05-01 11:52:22,932] A new study created in memory with name: no-name-01cc5f20-c957-4ac3-94fe-bf30ef58c14f
[I 2024-05-01 11:52:23,661] Trial 0 finished with value: 4.220970528315588 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'p': 7}. Best is trial 0 with value: 4.220970528315588.
[I 2024-05-01 11:52:23,768] Trial 1 finished with value: 5.357815074321426 and parameters: {'n_neighbors': 20, 'weights': 'uniform', 'p': 6}. Best is trial 0 with value: 4.220970528315588.
[I 2024-05-01 11:52:24,301] Trial 2 finished with value: 4.356868498617171 and parameters: {'n_neighbors': 14, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 4.220970528315588.
[I 2024-05-01 11:52:24,565] Trial 3 finished with value: 4.23182741590401 and parameters: {'n_neighbors': 2, 'weights': 'distance', 'p': 1}. Best is trial 0 with value: 4.220970528315588.
[I 2024-05-01 11:52:24,969] Trial 4 finished with value: 5.49884500250772 and parameters: {'n_neighbors': 19, 'weights': 'uniform'

Lagring av resultater

In [28]:
res_2_design = pd.DataFrame(res.dict_res_2_design)
res_2_design_opt = pd.DataFrame(res.dict_res_2_design_opt)

res_2_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_design.csv')
res_2_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_design_opt.csv')

Velger beste modell

# NIR målinger ekskludert

In [30]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt_NIR,
                             dict_res=res.dict_res_2_hel_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-05-01 12:30:13,422] A new study created in memory with name: no-name-2dd925a6-25b8-4d20-af2e-fbb06f8e8754
[I 2024-05-01 12:30:14,188] Trial 0 finished with value: 5.047108935995656 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'p': 8}. Best is trial 0 with value: 5.047108935995656.
[I 2024-05-01 12:30:14,195] Trial 1 finished with value: 5.625145252224232 and parameters: {'n_neighbors': 18, 'weights': 'uniform', 'p': 8}. Best is trial 0 with value: 5.047108935995656.
[I 2024-05-01 12:30:14,880] Trial 2 finished with value: 4.9640695884214665 and parameters: {'n_neighbors': 12, 'weights': 'distance', 'p': 6}. Best is trial 2 with value: 4.9640695884214665.
[I 2024-05-01 12:30:14,980] Trial 3 finished with value: 4.994693951764754 and parameters: {'n_neighbors': 12, 'weights': 'distance', 'p': 5}. Best is trial 2 with value: 4.9640695884214665.
[I 2024-05-01 12:30:15,607] Trial 4 finished with value: 4.7509172100371595 and parameters: {'n_neighbors': 5, 'weights': 'dis

Lagring av resultater

In [ ]:
res_2_hel_NIR = pd.DataFrame(res.dict_res_2_hel_NIR)
res_2_hel_opt_NIR = pd.DataFrame(res.dict_res_2_hel_opt_NIR)

res_2_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_hel_NIR.csv')
res_2_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_hel_opt_NIR.csv')

Velger beste modell

## Designsett

## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt_NIR,
                             dict_res=res.dict_res_2_design_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Lagring av resultater

In [ ]:
res_2_design_NIR = pd.DataFrame(res.dict_res_2_design_NIR)
res_2_design_opt_NIR = pd.DataFrame(res.dict_res_2_design_opt_NIR)

res_2_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_design_NIR.csv')
res_2_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_design_opt_NIR.csv')

Velger beste modell